# 22 set 2018

- Corrigir o desaparecimento de candidatos aptos da lista de candidatos
- Revisar as queries de consulta para gerar o resumo

## Situação de Candidatura

### CADASTRADO

**Aguardando julgamento**
{ id: 8 }
Candidato cujo pedido ainda não foi julgado.

### APTO

**Deferido**
{ id: 2 }
Candidato regular, com dados e documentação completos, já apreciados pelo juiz eleitoral.

**Deferido com recurso**
{ id: 16 }
Candidato julgado regular e deferido; no entanto, houve interposição de recurso contra essa decisão e aguarda julgamento por instância superior.

**Pendente de julgamento**
{ id: 17 }
Candidato cujo pedido inicial ainda não foi apreciado pelo juiz eleitoral.

**Indeferido com recurso**
{ id: 4 }
Candidato julgado não regular por não atender as condições necessárias para o deferimento do registro, que interpôs recurso contra essa decisão e aguarda julgamento por instância superior.

**Cassado com recurso**
{ id: 18 }
Candidato que teve seu registro da candidatura cancelado pelo partido e que interpôs recurso, ainda não julgado.

**Cancelado com recurso**
{ id: 19 }
Candidato cujo pedido de registro foi cancelado pelo partido, que interpôs recurso contra essa decisão.

### INAPTO
	
**Cancelado**
{ id: 5 }
Candidato que teve seu registro da candidatura cancelado pelo partido.

**Cassado**
{ id: 10 }
Candidato que teve o seu registro cassado.

**Falecido**
{ id: 7 }
Candidato com registro cancelado automaticamente assim que conhecido o fato pelo juízo eleitoral (Res. TSE 23.455 art. 69).

**Indeferido**
{ id: 14 }
Candidato que não reuniu as condições necessárias ao registro.

**Não conhecimento do pedido**
{ id: 13 }
Candidato cujo pedido de registro não será apreciado pelo juiz eleitoral.

**Renúncia**
{ id: 6 }
Candidato que desistiu de concorrer ao cargo e cuja renúncia já se encontra homologada pelo juiz eleitoral.

Assim, `hot_dados_candidato` fica:

```sql

WHERE
    (candidatura.eleicao_id = 202 OR candidatura.eleicao_id = 203) AND
    (
      candidatura.situacao_candidatura_id IS NULL OR
      candidatura.situacao_candidatura_id IN (2, 4, 8, 16, 17, 18, 19)
    );

```

## Candidaturas

Trazer somente as candidatura de 1º turno em que o candidato não renunciou. `hot_candidaturas_candidato` fica:

```sql

WHERE
    (candidatura.resultado_candidatura_id != 6 OR candidatura.resultado_candidatura_id IS NULL) AND
    (candidatura.situacao_candidatura_id != 6) AND
    (candidatura.eleicao_id != 202 AND candidatura.eleicao_id != 203)

```

## Dados do Resumo

`numero_candidaturas` de `view_resumo_candidaturas` fica com a mesma condição da hot table de candidatura:

```sql

COUNT(IF(
  (candidatura.turno = 1)
  AND (candidatura.situacao_candidatura_id != 6)
  AND (candidatura.eleicao_id != 202 AND candidatura.eleicao_id != 203)
  , 1
  , NULL
)) as numero_candidaturas,

```

# Atualizar Candidaturas

In [2]:
import pandas as pd
import numpy as np
import requests
import sys
import os
import glob
import time

In [14]:
from utils.dataframe_to_database import dataframe_to_database
from utils.dataframe_to_file import dataframe_to_file
from utils.database_to_dataframe import database_to_dataframe
from utils.download import download
from utils.tse_data_headers import get_header
from utils.tse_migrar_dados_por_estado_2018 import migrar_dados

In [5]:
header_2018 = get_header(2018)

In [7]:
PREFIXO_ARQUIVO_CANDIDATURAS = 'consulta_cand_'
DIRETORIO_TEMPORARIO = os.path.join('../temp')

def gerar_dataset():
    tempo_inicio_ano = time.time()

    arquivos = PREFIXO_ARQUIVO_CANDIDATURAS + '2018' + '*.csv'
    pathes = os.path.join(DIRETORIO_TEMPORARIO, arquivos)
    arquivos_do_ano = sorted(glob.glob(pathes))

    df_global = pd.DataFrame()
    for arquivo_estado in arquivos_do_ano:
        print('Iniciando {}'.format(arquivo_estado))

        df_estado = pd.read_csv(
            arquivo_estado,
            sep = ';',
            header = 0,
            dtype = np.str,
            names = header_2018,
            encoding = 'iso-8859-1'
        )
        df_global = df_global.append(df_estado)

    return df_global

In [10]:
df = gerar_dataset()

Iniciando ../temp\consulta_cand_2018_AC.csv
Iniciando ../temp\consulta_cand_2018_AL.csv
Iniciando ../temp\consulta_cand_2018_AM.csv
Iniciando ../temp\consulta_cand_2018_AP.csv
Iniciando ../temp\consulta_cand_2018_BA.csv
Iniciando ../temp\consulta_cand_2018_BR.csv
Iniciando ../temp\consulta_cand_2018_CE.csv
Iniciando ../temp\consulta_cand_2018_DF.csv
Iniciando ../temp\consulta_cand_2018_ES.csv
Iniciando ../temp\consulta_cand_2018_GO.csv
Iniciando ../temp\consulta_cand_2018_MA.csv
Iniciando ../temp\consulta_cand_2018_MG.csv
Iniciando ../temp\consulta_cand_2018_MS.csv
Iniciando ../temp\consulta_cand_2018_MT.csv
Iniciando ../temp\consulta_cand_2018_PA.csv
Iniciando ../temp\consulta_cand_2018_PB.csv
Iniciando ../temp\consulta_cand_2018_PE.csv
Iniciando ../temp\consulta_cand_2018_PI.csv
Iniciando ../temp\consulta_cand_2018_PR.csv
Iniciando ../temp\consulta_cand_2018_RJ.csv
Iniciando ../temp\consulta_cand_2018_RN.csv
Iniciando ../temp\consulta_cand_2018_RO.csv
Iniciando ../temp\consulta_cand_

In [11]:
df

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO
0,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,257,EMPRESÁRIO,0,-1,#NULO#,N,N,-1,06007352120186010000
1,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,999,OUTROS,-1,-1,#NULO#,N,N,-1,06007586420186010000
2,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,BRANCA,999,OUTROS,0,-1,#NULO#,N,S,-1,06005412120186010000
3,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,999,OUTROS,0,-1,#NULO#,N,S,-1,06005931720186010000
4,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,503,FAXINEIRO,0,-1,#NULO#,N,N,-1,06004060920186010000
5,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,298,SERVIDOR PÚBLICO MUNICIPAL,0,-1,#NULO#,N,S,-1,06003576520186010000
6,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,999,OUTROS,0,-1,#NULO#,N,N,-1,06001653520186010000
7,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,999,OUTROS,0,-1,#NULO#,N,N,-1,06006226720186010000
8,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,395,BANCÁRIO E ECONOMIÁRIO,0,-1,#NULO#,N,S,-1,06005958420186010000
9,22/09/2018,14:01:02,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,999,OUTROS,0,-1,#NULO#,N,N,-1,06001385220186010000


In [12]:
dataframe_to_file(
    dataframe=df,
    origem_dados='tse',
    nome='candidaturas-2018'
)

`DELETE FROM candidatura WHERE eleicao_id = 202;`

`DELETE FROM candidatura WHERE eleicao_id = 203;`

In [16]:
migrar_dados(2018)

Iniciando ../temp\consulta_cand_2018_AC.csv
Total de candidaturas em 2018: 586
586 de 586
Tempo total desse estado: 4.53s 

Iniciando ../temp\consulta_cand_2018_AL.csv
Total de candidaturas em 2018: 461
461 de 461
Tempo total desse estado: 3.60s 

Iniciando ../temp\consulta_cand_2018_AM.csv
Total de candidaturas em 2018: 855
855 de 855
Tempo total desse estado: 6.80s 

Iniciando ../temp\consulta_cand_2018_AP.csv
Total de candidaturas em 2018: 663
663 de 663
Tempo total desse estado: 5.10s 

Iniciando ../temp\consulta_cand_2018_BA.csv
Total de candidaturas em 2018: 1196
1196 de 1196
Tempo total desse estado: 13.57s 

Iniciando ../temp\consulta_cand_2018_BR.csv
Total de candidaturas em 2018: 28
28 de 28
Tempo total desse estado: 0.35s 

Iniciando ../temp\consulta_cand_2018_CE.csv
Total de candidaturas em 2018: 915
915 de 915
Tempo total desse estado: 8.31s 

Iniciando ../temp\consulta_cand_2018_DF.csv
Total de candidaturas em 2018: 1258
1258 de 1258
Tempo total desse estado: 10.40s 

Ini

## Buscar fotos que faltam

In [17]:
df_candidatos_2018 = database_to_dataframe(
    tabela='candidatura',
    colunas_tabela=['candidato_id', 'sequencial_candidato', 'sigla'],
    condicao='LEFT JOIN estado e on candidatura.estado_id = e.id LEFT JOIN candidato on candidatura.candidato_id = candidato.id WHERE (eleicao_id = 202 OR eleicao_id = 203) AND candidato.foto IS NULL'
)
df_candidatos_2018['sigla'] = df_candidatos_2018.sigla.fillna('BR').astype(str)

df_candidatos_2018

,candidato_id,sequencial_candidato,sigla
0,401236,10000629875,AC
1,1011,10000629953,AC
2,1273367,10000629101,AC
3,957468,10000628780,AC
4,1273372,10000629098,AC
5,1273924,10000629895,AC
6,1273925,20000630049,AL
7,1273926,20000630010,AL
8,646358,20000630007,AL
9,4519,20000629959,AL


In [18]:
falhas = []
total = len(df_candidatos_2018)
df_fotos = pd.DataFrame()

for candidato in df_candidatos_2018.itertuples():
    sys.stdout.write('\rCandidato {} de {}'.format(candidato.Index, total))
    sys.stdout.flush()

    while True:
        try:
            link = 'http://divulgacandcontas.tse.jus.br/divulga/rest/v1/candidatura/buscar/2018/{}/2022802018/candidato/{}'.format(
                candidato.sigla,
                candidato.sequencial_candidato
            )
            request = requests.get(link)
            response = request.json()

            df_i = pd.DataFrame(data = {
                'candidato_id': [candidato.candidato_id],
                'foto': [response['fotoUrl']]
            })

            if df_i.empty:
                raise Exception(candidato.candidato_id)
            else:
                df_fotos = df_fotos.append(df_i)
                break
        except:
            falhas.append(candidato.candidato_id)

Candidato 292 de 293

In [19]:
df_fotos.index = df_fotos.reset_index().index

df_fotos

,candidato_id,foto
0,401236,http://divulgacandcontas.tse.jus.br/candidatur...
1,1011,http://divulgacandcontas.tse.jus.br/candidatur...
2,1273367,http://divulgacandcontas.tse.jus.br/candidatur...
3,957468,http://divulgacandcontas.tse.jus.br/candidatur...
4,1273372,http://divulgacandcontas.tse.jus.br/candidatur...
5,1273924,http://divulgacandcontas.tse.jus.br/candidatur...
6,1273925,http://divulgacandcontas.tse.jus.br/candidatur...
7,1273926,http://divulgacandcontas.tse.jus.br/candidatur...
8,646358,http://divulgacandcontas.tse.jus.br/candidatur...
9,4519,http://divulgacandcontas.tse.jus.br/candidatur...


## Atualizar fotos no banco de dados

In [20]:
import mysql.connector as mysql

BD_CONFIG = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',
    'database': 'laddres'
}

In [21]:
cnx = mysql.connect(**BD_CONFIG)
cursor = cnx.cursor()

total = len(df_fotos)
for candidato in df_fotos.itertuples():
    sys.stdout.write('\r{} de {}'.format(candidato.Index, total))
    sys.stdout.flush()

    query = (
        "UPDATE candidato "
        "SET "
        "foto = %(foto)s "
        "WHERE id = %(candidato_id)s"
    )
    atributos = {
        'foto': candidato.foto,
        'candidato_id': candidato.candidato_id
    }

    try:
        cursor.execute(query, atributos)
        cnx.commit()
    except:
        print(candidato)
        cursor.close()
        cnx.close()
        raise

cursor.close()
cnx.close()

292 de 293

## Download das fotos

In [22]:
falhas = []
total = len(df_fotos)

for candidato in df_fotos.itertuples():
    sys.stdout.write('\rCandidato {} de {}'.format(candidato.Index, total))
    sys.stdout.flush()

    try:
        download(
            url=candidato.foto,
            filename='../data/images/laddres_{}.jpg'.format(candidato.candidato_id)
        )
    except:
        falhas.append(candidato.candidato_id)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[█████████████████████████████████████████████

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
[█████████████████████████████████████████████